## Stocks & Transformer
Use transformer for time series forecasting

In [32]:
stocks="XLE XLF XLU XLI XLK XLV XLY XLP XLB XOP XHB XME IYR IBB GLD"

In [33]:
import yfinance as yf
data = yf.download(stocks, start="1995-01-01", end="2021-09-01")

[*********************100%***********************]  15 of 15 completed


In [79]:
data.head()

Adj Close                                                          \
                 GLD IBB IYR XHB        XLB        XLE        XLF        XLI   
Date                                                                           
1998-12-22       NaN NaN NaN NaN  12.579885  13.664221  12.131743  15.526074   
1998-12-23       NaN NaN NaN NaN  12.712011  13.948701  12.310627  15.797000   
1998-12-24       NaN NaN NaN NaN  13.004562  13.875285  12.391943  16.005400   
1998-12-28       NaN NaN NaN NaN  12.891315  13.801879  12.229322  16.088760   
1998-12-29       NaN NaN NaN NaN  13.127251  13.939519  12.359423  16.317997   

                                  ... Volume                                \
                  XLK        XLP  ...    XLE     XLF   XLI     XLK     XLP   
Date                              ...                                        
1998-12-22  24.713181  15.850309  ...  15200   55887   600  300500  150300   
1998-12-23  25.303598  16.233488  ...  67800   78784   900  338300   44000   
1998-12-24  25.207203  16.205442  ...  12300   43824  1400  243700   45500   
1998-12-28  25.279501  16.111994  ...  13500   51948  7400  309300   25400   
1998-12-29  25.351797  16.467131  ...  22000  100819  1400  123800  123500   

                                         
              XLU    XLV    XLY XME XOP  
Date                                     
1998-12-22   7900   5700   3700 NaN NaN  
1998-12-23  24100  18100  12000 NaN NaN  
1998-12-24  23800   4900   1600 NaN NaN  
1998-12-28   3100  15500   8600 NaN NaN  
1998-12-29  12000   5300   1100 NaN NaN  

[5 rows x 90 columns]

In [35]:
adj_close=data['Adj Close']

### Set up dataframe and train/validate/test splits
We reserve data afer 2020 as our test dataset and use 85%/15% split for train/validation

In [146]:
from sklearn.model_selection import train_test_split
import random
np.random.seed(1337)

In [150]:
etfs=adj_close[['XLB','XLE','XLF','XLI','XLK','XLP','XLU','XLV','XLY']]

# let's try keep the data from '2019-10-1' to '2019-12-31' as buffer and use anythng after '2020-01-01' as test
etfs.loc[:, 'split']=' '
etfs.loc[etfs.index > '2020-01-01', 'split']='test'
index = etfs[etfs.index < '2019-10-01'].index
l1=list(index)
np.random.shuffle(l1)
train_size = int(len(l1)*0.85)
etfs.loc[l1[:train_size],'split'] = 'train'
etfs.loc[l1[train_size+1:],'split'] = 'val'

C:\Users\davefang\.conda\envs\textmining\lib\site-packages\pandas\core\indexing.py:1667: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = value
C:\Users\davefang\.conda\envs\textmining\lib\site-packages\pandas\core\indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)
C:\Users\davefang\.conda\envs\textmining\lib\site-packages\pandas\core\indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFr

In [173]:
etfs_log_chg = np.log1p(etfs[etfs.columns[:-2]].pct_change())
etfs_log_chg['split'] =  etfs.split
etfs_log_chg = etfs_log_chg.iloc[1:,:]

In [176]:
# save csv files
etfs.to_csv('sector_etf.csv')
etfs_log_chg.to_csv('sector_etf_logchg.csv')

### Transformer